In [2]:
import json
import os
import re
import os.path as osp
import pandas as pd
import glob
import tqdm
##Classe com vários métodos de pre-processamento de texto em português criado pelo grupo F03
import utils.preprocessing_portuguese as preprossPT


In [3]:
df_labels = pd.read_csv('./resultado_m03_meta_classes_extraction/relacao_documentos_label_v2.csv')
# Mapeia labels em inteiros
df_labels['label_int'] = pd.factorize(df_labels['label'])[0]
df_labels.head()

,doc_id,final_meta-class,label,label_int
0,499fa518c7724a6e449c509b7d7bc819,ATA,ata dispensa licitacao,0
1,56a09c5d1d04cc95ada4d68ad22dcbd9,ATA,ata dispensa licitacao,0
2,6e9f35208290c1130fcccad47b50aa53,ATA,ata dispensa licitacao,0
3,669c029c5812ec9a31b9c211044157b4,ATA,ata dispensa licitacao,0
4,a22421dc45d623c9200c7e8fb1e6ca34,ATA,ata dispensa licitacao,0


In [ ]:
def ner_filter(page_text, named_entities):
    

In [42]:
def get_name(directory):
    return re.search("licitacoes-(.*)/", directory)[1].replace("-", "_")

def list_json_files_dir(city_dir, city_name):
    if city_name != 'itamarati':
        return glob.glob(os.path.join(city_dir, 'data', 'files_json', '*'))
    else:
        return glob.glob(os.path.join(city_dir, '*', 'data', 'files_json', '*'))

def read_files(file_dir):
    with open(file_dir) as f:
        lines = f.read() # lê o conteúdo (pode ser lido em um stream, se achar necessário)
        return json.loads(lines)
    
def preprocess_text(document, num_pages, city_name):
    return [limpeza_texto(page_content, city_name) for page_content in document['text_content'][:4]]

def merge_pages(document, num_pages):
    # retorna lista onde a cada posicao uma nova pagina e concatenada ao texto
    num_pages+=1
    return [" ".join(document['text_preprocessed'][0:num_pages]) for num_pages in range(1,num_pages)]

In [ ]:
"""
    Faz a leitura dos arquivos, aplica o pre-processamento e armazena em um dataframe
"""

base_path = "../data/*licitacoes*/"
cities_dir = glob.glob(base_path)
num_pages = 4

cities_docs = {}
df_document_content = pd.DataFrame(columns=['doc_id', 'city', 'file_dir', 'one_page', 'two_pages', 'three_pages', 'four_pages'])

for city_dir in cities_dir:
    city_name = get_name(city_dir)
    #if city_name == "bh":
    if city_name != "olaria":
        continue
    print("-"*100)
    print(city_name)
    # Lista os arquivos a serem lidos
    files_dir = list_json_files_dir(city_dir, city_name)
    # Faz a leitura dos arquivos
    for file_dir in tqdm.tqdm_notebook(files_dir):
        document = read_files(file_dir)
        # Verifica se foi possível extrair texto do documento
        if document['status'] == 'SUCCESS':
            # preprocessamento
            document['text_preprocessed'] = preprocess_text(document, num_pages, city_name)
            # gera variacoes do texto concatenando 1 a num_pages páginas
            page_content = merge_pages(document, num_pages)
            # gera linha a ser inserida no dataframe
            new_row = [document['file_id'], city_name, file_dir]
            new_row.extend(page_content)
            # insere nova linha no dataframe
            df_document_content.loc[len(df_document_content)] = new_row

In [44]:
document = read_files("../data/302-licitacoes-olaria/data/files_json/7087aaed7cb40a722df310e1dbd7d881.json")

In [47]:
text = " ".join(document['text_content'])
text

'PREFEITURA MUNICIPAL DE OLARIA \n2013-2016 \n \nNUMER VALOR  VL.  VL TL  VENC.  OBJETO  CONCEDENTE  CONT.  SIT. OBRA  OBS \nO  CONV  C. PART  CONV.  CORRE\nCONV  NTE \n          CONSTRUÇÃO PONTE QUINTILIANOS  SETOP    VIGAS LIBERADAS  OF GAB SUBSEAM \n  465/2016 \n130/2016 \n          CONSTRUÇÃO 2 BUEIROS PARQUE DE  DEP       OFÍCIO 129/2016 \nEXPOSIÇÕES  \n          REFORMA ESCOLA MUNICIPAL  S.E.E    PROJETO ENTREGUE   \nAUTUALPA DUQUE  S.E.E \n  501.655,03    501.655,03    AMPLIAÇÃO ESCOLA MUNICIPAL  PAR/FNDE    PROJETO ENTREGUE   \nAUTUALPA DUQUE  / PROJETO INCENDIO \nPROTOCOLADO \nCORPO B. \n          14 MATA BURRO  SETOP    CONCLUIDO  PRESTAÇÃO DE \n  CONTAS ENVIADA \n12/12/2016 \n139/2014  101.889,94  1.018,90  102.908,84  27/06/2017  POÇO ARTESIANO E REDE  SEDESE  16814 – 9  ORDEM DE SERVIÇO   \nDISTRIBUIÇÃO PACIENCIA  TRAVESSIA  40.000,00 \nSOCIAL \n852/2014  30.000,00    30.000,00  23/06/2017  AQUISIÇÃO DE VEÍCULO   S.E.S  17000 – 3  AGUARDAR REPASSE   \n1.500,00  2ª PARCELA 

In [49]:
text

'PREFEITURA MUNICIPAL DE OLARIA \n -  \n \nNUMER VALOR  VL.  VL TL  VENC.  OBJETO  CONCEDENTE  CONT.  SIT. OBRA  OBS \nO  CONV  C. PART  CONV.  CORRE\nCONV  NTE \n          CONSTRUÇÃO PONTE QUINTILIANOS  SETOP    VIGAS LIBERADAS  OF GAB SUBSEAM \n  465/  \n  \n          CONSTRUÇÃO 2 BUEIROS PARQUE DE  DEP       OFÍCIO 129/  \nEXPOSIÇÕES  \n          REFORMA ESCOLA MUNICIPAL  S.E.E    PROJETO ENTREGUE   \nAUTUALPA DUQUE  S.E.E \n  501.655,03    501.655,03    AMPLIAÇÃO ESCOLA MUNICIPAL  PAR/FNDE    PROJETO ENTREGUE   \nAUTUALPA DUQUE  / PROJETO INCENDIO \nPROTOCOLADO \nCORPO B. \n          14 MATA BURRO  SETOP    CONCLUIDO  PRESTAÇÃO DE \n  CONTAS ENVIADA \n  \n139/2014  101.889,94  1.018,90  102.908,84     POÇO ARTESIANO E REDE  SEDESE  16814 – 9  ORDEM DE SERVIÇO   \nDISTRIBUIÇÃO PACIENCIA  TRAVESSIA  40.000,00 \nSOCIAL \n852/2014  30.000,00    30.000,00     AQUISIÇÃO DE VEÍCULO   S.E.S  17000 – 3  AGUARDAR REPASSE   \n1.500,00  2ª PARCELA \n   49.935,67    49.935,67     AQUISIÇÃO EQUI

In [50]:
df = pd.read_csv("../data/NER_results/Olaria.csv", sep="\t")
df.head()

df["named_entities"] = df["named_entities"].apply(eval)

for file_name in df['file'].values:
    if file_name == '7087aaed7cb40a722df310e1dbd7d881':
        named_entities = {
            "PESSOA": [],
            "ORGANIZAÇÃO": [],
            "LOCAL": [],
            "TEMPO": [],
            "LEGISLAÇÃO": [],
            "JURISPRUDÊNCIA": [],
        }

        for entity in df.loc[df["file"] == file_name,"named_entities"].values[0]:
            if entity[0] in named_entities.keys():
                named_entities[entity[0]].append(entity[1])

        for key in named_entities.keys():
            named_entities[key] = set(named_entities[key])
            for term in named_entities[key]:
                print(term)
                text = text.replace(term, " ")

MATA BURRO SETOP
PONTE QUINTILIANOS SETOP 
 
 VIGAS LIBERADAS
FRANCISCO DO 
PRATA 
FUNASA 
 
 CADASTRADO 
SICONV
FUNASA 
 
 CADASTRADO 
SICONV 
 
PROPOS
PO?OS ARTESIANO VILA TOM? E 
PELEJA 
FUNASA 
 
 CADASTRADO 
SICONV
23/06/2017
12/12/2016
2013
27/06/2017
29/06/2017
1556/2014
130/2016
2016
01/12/2017
21/06/2018
